In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("us-cities-demographics.csv", sep=";")

In [3]:
df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

#df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [9]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temp = pd.read_csv(fname)
df_temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [10]:
df_recent_temp = df_temp[(df_temp.dt >= "2000-01-01") & (df_temp.Country == "United States")]

In [11]:
print(df_recent_temp.shape)
df_recent_temp.sample(10)

(42405, 7)


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
5562845,2003-02-01,-1.365,0.247,Overland Park,United States,39.38N,93.64W
2902162,2006-02-01,11.421,0.304,Hayward,United States,37.78N,122.03W
4149388,2008-02-01,17.911,0.307,Laredo,United States,28.13N,99.09W
5138061,2007-06-01,24.862,0.164,Nashville,United States,36.17N,87.51W
3330172,2000-08-01,27.386,0.274,Jacksonville,United States,29.74N,81.23W
8154494,2013-02-01,-3.801,0.360,Warren,United States,42.59N,82.91W
4305631,2009-07-01,26.321,0.338,Little Rock,United States,34.56N,91.46W
2375039,2005-08-01,23.511,0.180,Fort Wayne,United States,40.99N,85.21W
3330217,2004-05-01,24.377,0.384,Jacksonville,United States,29.74N,81.23W
2902237,2012-05-01,16.555,0.255,Hayward,United States,37.78N,122.03W


In [12]:
df_air = pd.read_csv("airport-codes_csv.csv")
df_air.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [13]:
df_air.type.unique()

array(['heliport', 'small_airport', 'closed', 'seaplane_base',
       'balloonport', 'medium_airport', 'large_airport'], dtype=object)

In [14]:
df_air[~df_air.iata_code.isna() & (df_air.iso_country=="US") & (df_air.type.isin(["large_airport"]))]\
                .sample(10)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
28544,KPBI,large_airport,Palm Beach International Airport,19.0,NaN,US,US-FL,West Palm Beach,KPBI,PBI,PBI,"-80.09559631347656, 26.68320083618164"
27389,KGUS,large_airport,Grissom Air Reserve Base,812.0,NaN,US,US-IN,Peru,KGUS,GUS,GUS,"-86.1520996094, 40.648101806599996"
28394,KORF,large_airport,Norfolk International Airport,26.0,NaN,US,US-VA,Norfolk,KORF,ORF,ORF,"-76.20120239257812, 36.89459991455078"
28369,KOMA,large_airport,Eppley Airfield,984.0,NaN,US,US-NE,Omaha,KOMA,OMA,OMA,"-95.894096, 41.3032"
26611,KDTW,large_airport,Detroit Metropolitan Wayne County Airport,645.0,NaN,US,US-MI,Detroit,KDTW,DTW,DTW,"-83.35340118408203, 42.212398529052734"
26391,KCID,large_airport,The Eastern Iowa Airport,869.0,NaN,US,US-IA,Cedar Rapids,KCID,CID,CID,"-91.71080017089844, 41.884700775146484"
28584,KPIT,large_airport,Pittsburgh International Airport,1203.0,NaN,US,US-PA,Pittsburgh,KPIT,PIT,PIT,"-80.23290253, 40.49150085"
26415,KCLT,large_airport,Charlotte Douglas International Airport,748.0,NaN,US,US-NC,Charlotte,KCLT,CLT,CLT,"-80.94309997558594, 35.2140007019043"
30028,KSTL,large_airport,St Louis Lambert International Airport,618.0,NaN,US,US-MO,St Louis,KSTL,STL,STL,"-90.370003, 38.748697"
29902,KSAT,large_airport,San Antonio International Airport,809.0,NaN,US,US-TX,San Antonio,KSAT,SAT,SAT,"-98.46980285644531, 29.533700942993164"


### CITIES DEMOGRAPHIC DATA QUALITY ASSESSMENT

In [16]:
print(df.shape)
df.info()

(2891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


In [17]:
for col in df.columns:
    print(f"{col} has {df[col].isnull().sum()} null values")

City has 0 null values
State has 0 null values
Median Age has 0 null values
Male Population has 3 null values
Female Population has 3 null values
Total Population has 0 null values
Number of Veterans has 13 null values
Foreign-born has 13 null values
Average Household Size has 16 null values
State Code has 0 null values
Race has 0 null values
Count has 0 null values


In [18]:
# Problem: some columns have null values

# solution: since this comes from the golden source, it is better to leave them as they are for now.
# Most columns have correct values
# The data types as shown above are also accurate.

In [19]:
# Looks like it is not on the level of city granularity

df['City'].unique().size

567

In [20]:
df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [21]:
# Problem: The race and race counts columns are repeated for each city and state.
# So we remove them from the table and create a lookup matching on city and state.

race_counts_lookup = df.loc[:, ["City", "State", "Race", "Count"]].copy()

In [22]:
race_counts_lookup.head()

,City,State,Race,Count
0,Silver Spring,Maryland,Hispanic or Latino,25924
1,Quincy,Massachusetts,White,58723
2,Hoover,Alabama,Asian,4759
3,Rancho Cucamonga,California,Black or African-American,24437
4,Newark,New Jersey,White,76402


In [61]:
race_counts_lookup = race_counts_lookup.drop_duplicates()
race_counts_lookup.shape

(2891, 4)

In [62]:
# Write race counts to file

# race_counts_lookup.to_csv("cleaned_data/race_counts.csv", index=False)

In [23]:
df_no_race = df.drop(["Race", "Count"], axis=1)
print(df_no_race.shape)

(2891, 10)


In [24]:
# There are 567 unique cities. So we investigate further to check why the numbers do not match

df_no_race = df_no_race.drop_duplicates()
df_no_race.shape

(596, 10)

In [25]:
# This shows that the combination of cities and states are unique.
# i.e. same city name, but different states

city_states = df_no_race.groupby(["City", "State"]).agg({"State Code":"count"})
city_states[city_states["State Code"] > 1]

,,State Code
City,State,


In [63]:
# Save cleaned demographics file to disk for ETL

# df_no_race.to_csv("cleaned_data/city_demographics_cleaned.csv", index=False)

### AIRPORT CODES DATA QUALITY ASSESSMENT

In [27]:
df_air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


In [28]:
for col in df_air.columns:
    nulls = df_air[col].isnull().sum()
    all_count = nulls + df_air[col].count()
    print(f"{col} has {round((nulls/all_count) * 100, 3)}% null values")

ident has 0.0% null values
type has 0.0% null values
name has 0.0% null values
elevation_ft has 12.721% null values
continent has 50.33% null values
iso_country has 0.448% null values
iso_region has 0.0% null values
municipality has 10.306% null values
gps_code has 25.502% null values
iata_code has 83.315% null values
local_code has 47.915% null values
coordinates has 0.0% null values


In [29]:
# A good number of null values however, we do not need most columns

df_air.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [30]:
# Also some airports  or strips may not have an iata_code but may have a local code.
# We look at the combination of both
# Roughly 20k have both codes empty

df_air[df_air['iata_code'].isnull() & df_air['local_code'].isnull()].shape

(20187, 12)

In [31]:
df_air['type'].value_counts()

small_airport     33965
heliport          11287
medium_airport     4550
closed             3606
seaplane_base      1016
large_airport       627
balloonport          24
Name: type, dtype: int64

In [32]:
air_code_nulls = df_air[df_air['iata_code'].isnull() & df_air['local_code'].isnull()]
air_code_nulls['type'].value_counts()

small_airport     13010
heliport           3645
closed             2866
medium_airport      604
seaplane_base        35
large_airport        23
balloonport           4
Name: type, dtype: int64

In [33]:
# We get the US state codes for later joining

df_air['state_code'] = df_air['iso_region'].apply(lambda x: x.split("-")[1] if x.startswith("US") else x)
df_air['state_code'].sample(5)

11824    PG-WPD
8468         PA
40356    RU-MOW
41379     CL-LR
22765     SR-SI
Name: state_code, dtype: object

In [53]:
us_airstrips = df_air[df_air['iso_country']=='US']

print(us_airstrips.shape)
us_airstrips.head()

(22757, 13)


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,state_code
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125",PA
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022",KS
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968",AK
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172",AL
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087",AR


In [54]:
for col in us_airstrips.columns:
    nulls = us_airstrips[col].isnull().sum()
    all_count = nulls + us_airstrips[col].count()
    print(f"{col} has {round((nulls/all_count) * 100, 3)}% null values")

ident has 0.0% null values
type has 0.0% null values
name has 0.0% null values
elevation_ft has 1.05% null values
continent has 99.996% null values
iso_country has 0.0% null values
iso_region has 0.0% null values
municipality has 0.448% null values
gps_code has 7.791% null values
iata_code has 91.128% null values
local_code has 6.684% null values
coordinates has 0.0% null values
state_code has 0.0% null values


In [60]:
# Writing cleaned version to file

#us_airstrips.loc[:, ["ident", "type", "name", "elevation", "iso_country", "iso_region", "municipality",
 #                    "gps_code", "iata_code", "local_code", "coordinates", "state_code"]]\
  #          .to_csv("cleaned_data/us_air_codes.csv", index=False)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


### TEMPERATURE DATA QUALITY ASSESSMENT

In [34]:
df_temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [35]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB


In [36]:
# We will not be needing temperatures from the 1700 - 1899

df_temp_useful = df_temp[df_temp['dt'] >= "1900-01-01"]
df_temp_useful.shape

(4791150, 7)

In [37]:
for col in df_temp_useful.columns:
    nulls = df_temp_useful[col].isnull().sum()
    all_count = nulls + df_temp_useful[col].count()
    print(f"{col} has {round((nulls/all_count) * 100, 3)}%  are null values. Null count: {nulls}")

dt has 0.0%  are null values. Null count: 0
AverageTemperature has 0.064%  are null values. Null count: 3070
AverageTemperatureUncertainty has 0.064%  are null values. Null count: 3070
City has 0.0%  are null values. Null count: 0
Country has 0.0%  are null values. Null count: 0
Latitude has 0.0%  are null values. Null count: 0
Longitude has 0.0%  are null values. Null count: 0


In [38]:
# Since the main column is the AverageTemperature, it makes no sense to keep rows with null AverageTemperature

df_temp_useful = df_temp_useful[~df_temp_useful['AverageTemperature'].isnull()]
df_temp_useful.shape

(4788080, 7)

In [39]:
# Temperature is currently on a monthly basis.
# Dataset looks good.

df_temp_useful.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
1874,1900-01-01,-0.989,0.588,Århus,Denmark,57.05N,10.33E
1875,1900-02-01,-2.799,0.882,Århus,Denmark,57.05N,10.33E
1876,1900-03-01,0.592,0.429,Århus,Denmark,57.05N,10.33E
1877,1900-04-01,4.630,0.417,Århus,Denmark,57.05N,10.33E
1878,1900-05-01,9.576,0.521,Århus,Denmark,57.05N,10.33E


In [52]:
# Save this cleaned version for the ETL pipeline

# df_temp_useful.to_csv("cleaned_data/GlobalTemperature_clean.csv", index=False)

### IMMIGRATION DATA QUALITY ASSESSMENT

In [64]:
# View the data

df_immig=spark.read.parquet("sas_data")

In [65]:
df_immig.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [66]:
df_immig.count()

3096313

In [67]:
df_immig.show(5)

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

In [ ]:
# As this is the main table, we use as is in the ETL

## GET CATEGORY DATA FROM SAS SHEET

In [41]:
def create_lookup_data(filename, column_name, output_file_name):
    with open(filename, 'r') as f:
        i94_lookup_file = f.readlines()
        i94_mode = [line.strip().replace("'", "").replace(" ", "",2).split("=") for line in i94_lookup_file]
        df = pd.DataFrame(i94_mode, columns=['Code',column_name])
        df.to_csv(output_file_name, index=False)

def create_lookup_data_tight(filename, column_name, output_file_name):
    with open(filename, 'r') as f:
        i94_lookup_file = f.readlines()
        i94_mode = [line.strip().replace("'", "").split("=") for line in i94_lookup_file]
        df = pd.DataFrame(i94_mode, columns=['Code',column_name])
        df.to_csv(output_file_name, index=False)

In [ ]:
# create_lookup_data('I94MODE.txt', 'Mode', 'I94MODE_output.csv')

In [ ]:
#create_lookup_data('I94COUNTRY.txt', 'Country', 'I94COUNTRY_output.csv')

In [ ]:
# create_lookup_data_tight('I94US_CITIES.txt', 'US_city', 'I94US_CITIES.csv' )

In [ ]:
# create_lookup_data('I94PORT.txt', 'Ports', 'I94PORTS.csv')

In [ ]:
# create_lookup_data('I94STATE_CODES.txt', 'US_State_Codes', 'I94STATE_CODES.csv')

In [ ]:
# create_lookup_data('I94VISA.txt', 'Visa_type', 'I94VISA.csv')

In [42]:
# Join Temperature to Country Codes lookup from immigration dataset to get enrich with Country codes which
# allows us join back to immigration fact table

immig_cc = pd.read_csv("I94COUNTRY_output.csv")
immig_cc.head()

,Code,Country
0,582,"MEXICO Air Sea, and Not Reported (I-94, no la..."
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA


In [43]:
immig_cc['Country'] = immig_cc['Country'].str.lower().str.strip()
df_temp_useful['Country'] = df_temp_useful['Country'].str.lower()

df_temp_country = df_temp_useful.merge(immig_cc, how="left", left_on="Country", right_on="Country")

In [44]:
immig_cc['Country'].unique()

array(['mexico air sea, and not reported (i-94, no land arrivals)',
       'afghanistan', 'albania', 'algeria', 'andorra', 'angola',
       'anguilla', 'antigua-barbuda', 'argentina', 'armenia', 'aruba',
       'australia', 'austria', 'azerbaijan', 'bahamas', 'bahrain',
       'bangladesh', 'barbados', 'belgium', 'belize', 'benin', 'bermuda',
       'belarus', 'bhutan', 'bolivia', 'bonaire, st eustatius, saba',
       'bosnia-herzegovina', 'botswana', 'brazil',
       'british virgin islands', 'brunei', 'bulgaria', 'burkina faso',
       'burma', 'burundi', 'cameroon', 'cape verde', 'cayman islands',
       'central african republic', 'chad', 'chile', 'china, prc',
       'curacao', 'christmas island', 'cocos islands', 'colombia',
       'comoros', 'congo', 'cook islands', 'costa rica', 'croatia', 'cuba',
       'cyprus', 'czech republic', 'faroe islands (part of denmark)',
       'denmark', 'djibouti', 'dominica', 'dominican republic',
       'east timor', 'ecuador', 'egypt', 'el salv

In [45]:
# some countries do not get a code.

print(df_temp_country.shape)
print(df_temp_country['Code'].isnull().sum())


df_temp_country.sample(10)

(4788080, 8)
1098401


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,Code
1003069,1936-10-01,28.428,0.522,Cuiabá,brazil,15.27S,55.83W,689.0
477090,1985-05-01,12.308,0.247,Besançon,france,47.42N,5.92E,111.0
3526909,1954-12-01,-5.006,0.343,Reykjavík,iceland,65.09N,21.06W,115.0
2885079,2011-08-01,26.679,0.221,Nazilli,turkey,37.78N,28.47E,264.0
1871963,1932-01-01,18.835,0.532,Jamshedpur,india,23.31N,86.50E,213.0
3450641,1964-12-01,16.869,0.179,Quito,ecuador,0.80S,77.95W,692.0
4694064,1908-09-01,20.250,0.342,Yonago,japan,36.17N,133.26E,209.0
4763240,1989-09-01,28.126,0.327,Zhuhai,china,21.70N,113.37E,NaN
943439,1970-01-01,17.375,0.419,Colima,mexico,20.09N,104.08W,NaN
4689664,1996-09-01,23.351,0.298,Yizheng,china,32.95N,119.68E,NaN


In [46]:
df_temp_country[df_temp_country['Code'].isnull()]['Country'].unique()

array(['china', 'canada', "côte d'ivoire", 'united states', 'mexico',
       'congo (democratic republic of the)', 'bosnia and herzegovina',
       'guinea bissau', 'puerto rico', 'cambodia'], dtype=object)

In [47]:
def clean_country_join_column(x):
    lookup = {'china, prc':'china',
              'ivory coast': "côte d'ivoire",
              'mexico air sea, and not reported (i-94, no land arrivals)':'mexico',
              'bosnia-herzegovina': 'bosnia and herzegovina',
              'guinea-bissau': 'guinea bissau'}
    
    return lookup[x] if x in lookup else x


immig_cc['Country_for_join'] = immig_cc['Country'].apply(clean_country_join_column)

In [48]:
# We join again

df_temp_country = df_temp_useful.merge(immig_cc, how="left", left_on="Country", right_on="Country_for_join")

In [49]:
# Some countries like Cambodia, United States and Puerto Rico do not appear in the immigration data

print(df_temp_country.shape)
print(df_temp_country['Code'].isnull().sum())

(4788080, 10)
421760


In [50]:
df_temp_country.sample(10)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country_x,Latitude,Longitude,Code,Country_y,Country_for_join
2085689,2010-02-01,-1.205,0.378,Kelowna,canada,49.03N,120.00W,NaN,NaN,NaN
610706,1980-07-01,15.651,0.233,Breda,netherlands,52.24N,5.26E,123.0,netherlands,netherlands
3007592,1989-11-01,12.812,0.253,Oakland,united states,37.78N,122.03W,NaN,NaN,NaN
517912,1977-03-01,26.739,0.570,Bihar,india,24.92N,85.12E,213.0,india,india
264952,1926-06-01,18.140,0.468,Aurora,united states,39.38N,104.05W,NaN,NaN,NaN
1627192,1995-10-01,17.631,0.140,Himeji,japan,34.56N,134.27E,209.0,japan,japan
2062106,1977-03-01,14.039,0.582,Kathmandu,nepal,28.13N,84.55E,257.0,nepal,nepal
2327095,2006-10-01,-0.722,0.217,Lasa,china,29.74N,90.46E,245.0,"china, prc",china
1483865,1988-01-01,24.496,0.294,Guarulhos,brazil,23.31S,46.31W,689.0,brazil,brazil
1284767,1994-01-01,-9.488,0.320,Flint,united states,42.59N,82.91W,NaN,NaN,NaN


In [51]:
# These do not appear in the immigration set so there is not much we can do about them

df_temp_country[df_temp_country['Code'].isnull()]['Country_x'].unique()

array(['canada', 'united states', 'congo (democratic republic of the)',
       'puerto rico', 'cambodia'], dtype=object)

In [ ]:
# See etl.ipynb for ETL